# 3rd Model

The process is:
1. Clean up data
    The process is remove Objects features (Categorical Values) that we could convert
    Note. The features were removed since the get_dummy function failed converting values
    We include features based on a Random Forest model (Feature of Importance) 
    Removed Null Values with the MODE values
2. Create Model
    1 - This notebook contains Logistic Regression (sklearn)
    2 - This notebook contains Keras Sequencial Model (TensorFlow)
3. Train
2. Predict (Stats)

In [1]:
import warnings
warnings.simplefilter('ignore')

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
#chunksize returns as a loop through the dataset. 
def transform_data(method, df_chunk):
    chunk_final = pd.DataFrame()
    if method is None:
        for chunk in df_chunk:
            chunk_final = chunk_final.append(chunk)
    else:
        if isinstance(df_chunk, pd.io.parsers.TextFileReader):
            for chunk in df_chunk:
                chunk_mod = method(chunk)
                chunk_final = chunk_final.append(chunk_mod)
        if isinstance(df_chunk, list):
            for chunk in df_chunk:
                chunk_mod = method(chunk)
                chunk_final = chunk_final.append(chunk_mod)
    return chunk_final

In [3]:
def cleanup_data(data_frame):
    print("Start cleanup_data")
    for column in data.columns:
        if data_frame[column].isnull().sum() > 0:
            print(column)
            data_frame[column].fillna(data_frame[column].mode().iloc[0], inplace=True)
    #data_frame = data_frame.fillna(data_frame.mean())
    #data_frame = data_frame.fillna(0)
    return data_frame

In [4]:
#def get_X(dataframe):
#    print("start get_X")
#    X = dataframe.drop("HasDetections", axis=1)
#    X = pd.get_dummies(X, 
#                   columns=["MachineIdentifier", "ProductName", "EngineVersion", "AppVersion", "AvSigVersion", "Platform", "Processor", "OsVer", "OsPlatformSubRelease", "OsBuildLab", 
#                            "SkuEdition", "PuaMode", "SmartScreen", "Census_MDC2FormFactor", "Census_DeviceFamily", "Census_ProcessorClass", "Census_PrimaryDiskTypeName", "Census_ChassisTypeName", "Census_PowerPlatformRoleName", 
#                            "Census_InternalBatteryType", "Census_OSVersion", "Census_OSArchitecture", "Census_OSBranch", "Census_OSEdition", "Census_OSSkuName", "Census_OSInstallTypeName", 
#                            "Census_OSWUAutoUpdateOptionsName", "Census_GenuineStateName", "Census_ActivationChannel", "Census_FlightRing"], drop_first=True)
#    return X

def get_X(dataframe): 
    print("start get_X")
    print(dataframe.shape)
    dataframe = dataframe[['ProductName','CountryIdentifier','CityIdentifier', 'Platform', 'Processor', 'SkuEdition', 'IsProtected',
                          'Firewall','Census_MDC2FormFactor', 'Census_ProcessorCoreCount', 'Census_PrimaryDiskTotalCapacity',
                          'Census_PrimaryDiskTypeName', 'Census_TotalPhysicalRAM', 'Census_InternalBatteryType','Census_OSArchitecture',
                          'Census_OSInstallLanguageIdentifier','Wdft_IsGamer']]
    
    columns = ['ProductName','CountryIdentifier','Platform','Processor', 'SkuEdition', 'IsProtected',
                          'Firewall','Census_MDC2FormFactor', 'Census_ProcessorCoreCount',
                          'Census_PrimaryDiskTypeName', 'Census_InternalBatteryType','Census_OSArchitecture',
                          'Census_OSInstallLanguageIdentifier','Wdft_IsGamer']
    
    for column in columns:
        print(column, dataframe[column].dtype, len(dataframe[column].unique()))
        print(dataframe.shape)
        dataframe = pd.get_dummies(dataframe, columns=[column], drop_first=True)
        print(dataframe.shape)
    
    return dataframe

In [5]:
#df_chunk = pd.read_csv('Resources/train.csv', chunksize=500000)
data = pd.read_csv('Resources/train.csv.gz', compression="gzip")
#data = transform_data(None, df_chunk)
print(data.shape)

(8921483, 83)


In [6]:
y = data[["HasDetections"]]
print(y.shape)

(8921483, 1)


In [7]:
data = get_X(data)
data.head()

start get_X
(8921483, 83)
ProductName object 6
(8921483, 17)
(8921483, 21)
CountryIdentifier int64 222
(8921483, 21)
(8921483, 241)
Platform object 4
(8921483, 241)
(8921483, 243)
Processor object 3
(8921483, 243)
(8921483, 244)
SkuEdition object 8
(8921483, 244)
(8921483, 250)
IsProtected float64 3
(8921483, 250)
(8921483, 250)
Firewall float64 3
(8921483, 250)
(8921483, 250)
Census_MDC2FormFactor object 13
(8921483, 250)
(8921483, 261)
Census_ProcessorCoreCount float64 46
(8921483, 261)
(8921483, 304)
Census_PrimaryDiskTypeName object 5
(8921483, 304)
(8921483, 306)
Census_InternalBatteryType object 79
(8921483, 306)
(8921483, 382)
Census_OSArchitecture object 3
(8921483, 382)
(8921483, 383)
Census_OSInstallLanguageIdentifier float64 40
(8921483, 383)
(8921483, 420)
Wdft_IsGamer float64 3
(8921483, 420)
(8921483, 420)


,CityIdentifier,Census_PrimaryDiskTotalCapacity,Census_TotalPhysicalRAM,ProductName_mse,ProductName_mseprerelease,ProductName_scep,ProductName_win8defender,ProductName_windowsintune,CountryIdentifier_2,CountryIdentifier_3,...,Census_OSInstallLanguageIdentifier_31.0,Census_OSInstallLanguageIdentifier_32.0,Census_OSInstallLanguageIdentifier_33.0,Census_OSInstallLanguageIdentifier_34.0,Census_OSInstallLanguageIdentifier_35.0,Census_OSInstallLanguageIdentifier_36.0,Census_OSInstallLanguageIdentifier_37.0,Census_OSInstallLanguageIdentifier_38.0,Census_OSInstallLanguageIdentifier_39.0,Wdft_IsGamer_1.0
0,128035.0,476940.0,4096.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1482.0,476940.0,4096.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,153579.0,114473.0,4096.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,20710.0,238475.0,4096.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,37376.0,476940.0,6144.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
data = cleanup_data(data)

Start cleanup_data
CityIdentifier
Census_PrimaryDiskTotalCapacity
Census_TotalPhysicalRAM


In [9]:
print(data.shape, y.shape)

(8921483, 420) (8921483, 1)


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, random_state=1, stratify=y)
print(X_train.shape, y_train.shape)

(6691112, 420) (6691112, 1)


In [11]:
print(X_train.shape, y_train.shape)

(6691112, 420) (6691112, 1)


In [12]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
predict_train = classifier.predict(X_train)
predict_test = classifier.predict(X_test)

In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_train, predict_train))
print(classification_report(y_test, predict_test))

             precision    recall  f1-score   support

          0       0.53      0.45      0.49   3346943
          1       0.52      0.60      0.56   3344169

avg / total       0.52      0.52      0.52   6691112

             precision    recall  f1-score   support

          0       0.53      0.45      0.49   1115648
          1       0.52      0.60      0.56   1114723

avg / total       0.52      0.52      0.52   2230371



In [12]:
X_train.shape[1]

420

In [13]:
y_train["HasDetections"].head()

7433569    0
4610591    0
8339454    1
5962255    1
7407408    0
Name: HasDetections, dtype: int64

In [14]:
y_train["HasDetections"] = y_train["HasDetections"].apply(lambda x: "True" if x==1 else "False")
y_train.head()

,HasDetections
7433569,False
4610591,False
8339454,True
5962255,True
7407408,False


In [15]:
Y_train = pd.get_dummies(y_train)
Y_train.head()

,HasDetections_False,HasDetections_True
7433569,1,0
4610591,1,0
8339454,0,1
5962255,0,1
7407408,1,0


In [16]:
y_test["HasDetections"].head()

3164226    1
556644     1
7764601    0
3856066    0
8261251    0
Name: HasDetections, dtype: int64

In [17]:
y_test["HasDetections"] = y_test["HasDetections"].apply(lambda x: "True" if x==1 else "False")
y_test.head()

,HasDetections
3164226,True
556644,True
7764601,False
3856066,False
8261251,False


In [18]:
Y_test = pd.get_dummies(y_test)
Y_test.head()

,HasDetections_False,HasDetections_True
3164226,0,1
556644,0,1
7764601,1,0
3856066,1,0
8261251,1,0


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Activation
output_dim = nb_classes = 2
input_dim = X_train.shape[1]
model = Sequential()
model.add(Dense(output_dim, input_dim=input_dim, activation="softmax"))
batch_size = 256
nb_epoch = 10

Using TensorFlow backend.


In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 842       
Total params: 842
Trainable params: 842
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

Train on 6691112 samples, validate on 2230371 samples
Epoch 1/10
6691112/6691112 [==============================] - 184s 27us/step - loss: 8.0564 - acc: 0.5002 - val_loss: 8.0571 - val_acc: 0.5001
Epoch 2/10
6691112/6691112 [==============================] - 142s 21us/step - loss: 8.0572 - acc: 0.5001 - val_loss: 8.0571 - val_acc: 0.5001
Epoch 3/10
6691112/6691112 [==============================] - 140s 21us/step - loss: 8.0572 - acc: 0.5001 - val_loss: 8.0571 - val_acc: 0.5001
Epoch 4/10
6691112/6691112 [==============================] - 141s 21us/step - loss: 8.0572 - acc: 0.5001 - val_loss: 8.0571 - val_acc: 0.5001
Epoch 5/10
6691112/6691112 [==============================] - 140s 21us/step - loss: 8.0572 - acc: 0.5001 - val_loss: 8.0571 - val_acc: 0.5001
Epoch 6/10
1470464/6691112 [=====>........................] - ETA: 1:40 - loss: 8.0521 - acc: 0.5004

In [21]:
import sys
default_stdout = sys.stdout
sys.stdout = open('keras_training.txt', 'w')

model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=2, validation_data=(X_test, Y_test))

sys.stdout = sys.__stdout__
sys.stdout = default_stdout

In [22]:
train_model_loss, train_model_accuracy = model.evaluate(X_train, Y_train, batch_size=256)
test_model_loss, test_model_accuracy = model.evaluate(X_test, Y_test, batch_size=256)
#print(f"Training Data Score: {training_r2}")
#print(f"Testing Data Score: {test_r2}")
print(f"Train Loss: {train_model_loss}, Accuracy: {train_model_accuracy}")
print(f"Test Loss: {test_model_loss}, Accuracy: {test_model_accuracy}")

2230371/2230371 [==============================] - 14s 6us/step
Train Loss: 8.055706690459415, Accuracy: 0.5002072899092891
Test Loss: 8.05570547968895, Accuracy: 0.5002073646050054


In [23]:
predict_train = model.predict(X_train)
predict_test = model.predict(X_test)

In [24]:
print(predict_train.shape, Y_train.shape)
print(predict_test.shape, Y_test.shape)

(6691112, 2) (6691112, 2)
(2230371, 2) (2230371, 2)


In [40]:
#model.model.save("secuencial_trained_2.h5")

In [41]:
from keras.models import load_model
model_load = load_model("secuencial_trained_2.h5")

In [42]:
model_loss, model_accuracy = model_load.evaluate(X_test, Y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 8.062390107589358, Accuracy: 0.4997926353956627


In [ ]:
#plt.scatter(predict_train, predict_train - Y_train, c="blue", label="Training Data")
plt.scatter(predict_train[0], c="blue", label="Training Data")
#plt.scatter(predict_test, predict_test - Y_test, c="orange", label="Testing Data")
#plt.scatter(predict_test, c="orange", label="Testing Data")
plt.legend()
#plt.hlines(y=0, xmin=y_test.min(), xmax=y_test.max())
plt.title("Residual Plot")
plt.show()